In [66]:
import pandas as pd

### Download current Checkin CSV from the [checkin website](https://msc-registration-checkin.onrender.com/)

In [67]:
import pandas as pd

df = pd.read_csv('./MSC-Checkin.csv')

### Total # of People + Families Checked In

In [68]:
# Count checked in people
checked_in = df[df['checkin'] == 'checked-in']
num_checked_in = len(checked_in)

print(f"Number of people checked in: {num_checked_in}")
print("\nFamily IDs of checked in people:")
print(checked_in['Family ID'].unique())
print("\nNumber of unique families checked in:")
print(len(checked_in['Family ID'].unique()))

Number of people checked in: 333

Family IDs of checked in people:
[ 32  37   1   2   3  10  42  45  55  64  69  71  72  73  75  77  80  81
  84  85  86  91  92  98  99 100 104 106 107 118 123 125 127 128 129 131
 139 141 144 145 147 152 155 163 169 181 182 184 189 191 199 211 215 217
 221 222 225 230 239 240 244 245 251 260 261 262 263 267 268 269 275 276
 281 282 289 294 295 296 301 303 306 308 317 318 323 330 332 334 335 339
 359 360 369 373 374 376 380 381 383 384 386 387 389 391 393 394 395 397
 398 399 402 411 433 434 440 443 444 463 464 465 468 472 478 479 484 486
 489 495 507 512 532 540 541 546 554 583 596 600 606 608 611 613 616 655
 657 660 676 682 684 717 718]

Number of unique families checked in:
151


### Checkin Count by Gender

In [69]:
# Get gender counts
gender_counts = checked_in['Participants.gender'].value_counts()
num_males = gender_counts.get('M', 0)
num_females = gender_counts.get('F', 0)

# Get adult count
num_adults = len(checked_in[checked_in['Category'] == 'Adult'])

# Get counts by grade/age
grade_counts = checked_in[checked_in['Category'].astype(str).str.isnumeric()]['Category'].value_counts().sort_index()

print(f"\nChecked-in Statistics:")
print(f"Total checked in: {num_checked_in}")
print(f"Males checked in: {num_males}")
print(f"Females checked in: {num_females}")
print(f"Adults checked in: {num_adults}")
# Define grade order
grade_order = ['Infant/ShishuVihar', 'Pre-KG', 'KG'] + [str(i) for i in range(1,13)]

# Convert grades to categorical with custom order
grade_counts = checked_in[checked_in['Category'].astype(str).str.isnumeric()]['Category']
grade_counts = grade_counts.astype('category').cat.set_categories(grade_order, ordered=True)
grade_counts = grade_counts.value_counts().sort_index()

print("\nChildren checked in by grade/age:")
print(grade_counts.to_string())



Checked-in Statistics:
Total checked in: 333
Males checked in: 141
Females checked in: 192
Adults checked in: 283

Children checked in by grade/age:
Infant/ShishuVihar     0
Pre-KG                 0
KG                     0
1                      1
2                      1
3                      1
4                      7
5                      1
6                      3
7                      5
8                      7
9                     11
10                     6
11                     1
12                     1


### Delete old `checkin_statistics.xlsx` file

In [70]:
# Delete checkin_statistics.xlsx if it exists
import os
if os.path.exists('checkin_statistics.xlsx'):
    os.remove('checkin_statistics.xlsx')


### Update with new `checkin_statistics.xlsx` file

In [71]:
# Get gender counts
gender_counts = checked_in['Participants.gender'].value_counts()
num_males = gender_counts.get('M', 0)
num_females = gender_counts.get('F', 0)

# Get adult count
num_adults = len(checked_in[checked_in['Category'] == 'Adult'])
adult_males = len(checked_in[(checked_in['Category'] == 'Adult') & (checked_in['Participants.gender'] == 'M')])
adult_females = len(checked_in[(checked_in['Category'] == 'Adult') & (checked_in['Participants.gender'] == 'F')])

# Get counts by grade/age
grade_counts = checked_in[checked_in['Category'].astype(str).str.isnumeric()]['Category'].value_counts().sort_index()

# Define grade order and get grade counts
grade_order = ['Infant/ShishuVihar', 'Pre-KG', 'KG'] + [str(i) for i in range(1,13)]
grade_counts = checked_in[checked_in['Category'].astype(str).str.isnumeric()]['Category']
grade_counts = grade_counts.astype('category').cat.set_categories(grade_order, ordered=True)
grade_counts = grade_counts.value_counts().sort_index()

# Get gender counts by grade
grade_gender_counts = {}
for grade in grade_counts.index:
    grade_gender_counts[grade] = {
        'M': len(checked_in[(checked_in['Category'] == grade) & (checked_in['Participants.gender'] == 'M')]),
        'F': len(checked_in[(checked_in['Category'] == grade) & (checked_in['Participants.gender'] == 'F')])
    }

# Create summary statistics dataframe with children by grade included
summary_stats = pd.DataFrame({
    'Metric': ['Total Checked In', 'Males Checked In', 'Females Checked In', 'Adults Checked In'] + 
             [f'Grade {grade}' for grade in grade_counts.index],
    'Male': [num_males, num_males, 0, adult_males] + 
           [grade_gender_counts[grade]['M'] for grade in grade_counts.index],
    'Female': [num_females, 0, num_females, adult_females] + 
             [grade_gender_counts[grade]['F'] for grade in grade_counts.index],
    'Count': [num_checked_in, num_males, num_females, num_adults] + 
             list(grade_counts.values)
})

# Export to Excel
with pd.ExcelWriter('checkin_statistics.xlsx') as writer:
    summary_stats.to_excel(writer, sheet_name='Statistics', index=False)

print("Statistics exported to checkin_statistics.xlsx")

Statistics exported to checkin_statistics.xlsx
